In [1]:
import pandas
#dataframe = pandas.read_csv('prelim_data.csv')
store = pandas.HDFStore('store.hdf5')
dataframe = store['dataframe']
#store['dataframe'] = dataframe
#store.close()

In [2]:
! wget http://flystocks.bio.indiana.edu/bloomington.csv

--2015-12-06 20:19:59--  http://flystocks.bio.indiana.edu/bloomington.csv
Resolving flystocks.bio.indiana.edu (flystocks.bio.indiana.edu)... 134.68.93.37
Connecting to flystocks.bio.indiana.edu (flystocks.bio.indiana.edu)|134.68.93.37|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11828650 (11M) [text/plain]
Saving to: ‘bloomington.csv.5’

100%[======================================>] 11,828,650  4.75MB/s   in 2.4s   

2015-12-06 20:20:02 (4.75 MB/s) - ‘bloomington.csv.5’ saved [11828650/11828650]



In [3]:
from datetime import date
from random import randint

from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_notebook,output_file, show, vform

output_notebook()
data = dict(
        dates=[date(2014, 3, i+1) for i in range(10)],
        downloads=[randint(0, 100) for i in range(10)],
    )
source = ColumnDataSource(dataframe)

columns = [TableColumn(title = x,field = x) for x in dataframe]
data_table = DataTable(source=source,fit_columns = True,columns = columns,width=950, height=280,editable = True)

show(vform(data_table))

ImportError: No module named models

In [4]:
not_screened = dataframe[dataframe['Expression data'].isin(['N'])]

In [5]:
order_8_13_15 = ['GMR49E02','GMR59H06','GMR69A03','GMR75B10','GMR75C06',
                 'GMR79C08','GMR83A06','GMR84D01','GMR30A09','GMR35G07',
                 'GMR46D02','GMR71D08']

In [6]:
not_screened[~not_screened['GenotypeKey'].isin(order_8_13_15)]

,Bloomington ID,FlyranchAddress,GenotypeKey,Priority,Expression data,Comment1,Rational
3,41329,NaN,GMR9F09,2,N,NaN,Cesar Mendes leg driver
7,39854,NaN,GMR74D08,NaN,N,X J-219,Cesar Mendes leg driver
12,45484,NaN,GMR23B03,2,N,NaN,Cesar Mendes leg driver
13,47241,NaN,GMR93H09,2,N,NaN,Cesar Mendes leg driver
14,47286,NaN,GMR95D12,NaN,N,X J-64,Cesar Mendes leg driver
16,47714,NaN,GMR67D05,2,N,NaN,Cesar Mendes leg driver
17,48020,NaN,GMR9B09,2,N,NaN,Cesar Mendes leg driver
18,49411,NaN,GMR65C04,2,N,NaN,Cesar Mendes leg driver
20,49907,NaN,GMR35D04,2,N,NaN,Cesar Mendes leg driver
22,50300,NaN,GMR47C12,2,N,NaN,Cesar Mendes leg driver


In [7]:
#find the lines that have not been screened
GMR_ids = not_screened[~not_screened['GenotypeKey'].isin(order_8_13_15)]['GenotypeKey']
GMR_ids = [x[-5:] for x in GMR_ids]
GMR_ids = [x for x in GMR_ids if not(')' in x)]

In [8]:
#load the strain database
strain_db = pandas.read_csv('strain_db.csv')

In [9]:
#parse the 'nickname' field to create a list that contains the striped GMR id or the string 'none'
import numpy as np
db_GMR_lines = [str(x).find('R') == 0 for x in strain_db['nickname']]
#search for candidate lines that me might already have in the incubator
import string
db_already_have = strain_db['nickname'].str.contains(string.join(GMR_ids,'|'),na=False)

In [10]:
#these are the lines we might already have
strain_db['nickname'][db_already_have]

75     R58H10-Gal4 (III);\nUAS-GCaMP6s (II)
77     R74D08-Gal4 (III);\nUAS-GCaMP6s (II)
254                       R95D12-Gal4 (III)
385                       R58H10-Gal4 (III)
409                        R74D08-Gal4 (II)
425                       R27H05_GAL4 (III)
773                    105D03-AD; 58H10-DBD
777                    116E03-AD; 58H10-DBD
779                    120F09-AD; 58H10-DBD
Name: nickname, dtype: object

In [11]:
#create a dictonary of the number of hits in the strain_db for each target line
target_dict = dict()
for target_line in GMR_ids:
    num_hits = 0
    for test_line in strain_db['nickname']:
        num_hits += (np.str(test_line).find(target_line) > 0)
    target_dict[target_line] = num_hits

In [12]:
#the lines that are NOT in the straindb
not_in_db = set([key for key,value in target_dict.items() if value ==0])
ordered_set = set([x[3:] for x in order_8_13_15])
already_ordered = not_in_db.intersection(ordered_set)
order_me = not_in_db - already_ordered

In [13]:
already_ordered

{'35G07', '59H06', '71D08'}

In [14]:
order_me

{'23B03',
 '35D04',
 '47C12',
 '5 E08',
 '65C04',
 '67D05',
 '68D03',
 '88F07',
 '93H09',
 'R9B09',
 'R9F09'}

In [15]:
#parse the bloomington stocklist
try:
    bloomington = pandas.read_csv('bloomington.csv')
except Exception as err:
    import csv
    with open('bloomington.csv', 'rt') as csvfile:
        spamreader = csv.reader(csvfile,skipinitialspace=True)
        bloomlist = [l for l in spamreader]
        num_colls = len(bloomlist[0])
        lines = list()
        for line in bloomlist[1:-1]:
            parsed_list = line[:num_colls]
            parsed_list[-1] = parsed_list[-1] + ''.join(parsed_list[num_colls:-1])
            lines.append(parsed_list)
        bloomington = pandas.DataFrame(lines[1:],columns = bloomlist[0])

In [16]:
def search_table(datatable,search_str):
    found_idx = list()
    for col in datatable.columns:
        found_idx.append(datatable[col].str.contains(search_str))
    found_idx = np.sum(found_idx,axis = 0)>0
    return found_idx

In [17]:
order_masks = [search_table(bloomington,order_str) for order_str in order_me]
bloomington[np.sum(order_masks,axis = 0)>0]

,Stk #,Genotype,Ch # all,Ch TE{},Breakpts/Insertion,A.K.A.,Date added,Donor,Donor's source,Comments
37407,41329,w[1118]; P{y[+t7.7] w[+mC]=GMR9F09-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,9/19/2012,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
41346,45484,w[1118]; P{y[+t7.7] w[+mC]=GMR23B03-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,4/5/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
42397,47241,w[1118]; P{y[+t7.7] w[+mC]=GMR93H09-GAL4}attP2...,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,8/16/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
42709,47714,w[1118]; P{y[+t7.7] w[+mC]=GMR67D05-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,11/11/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
42946,48020,w[1118]; P{y[+t7.7] w[+mC]=GMR9B09-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,11/18/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
44281,49411,w[1118]; P{y[+t7.7] w[+mC]=GMR65C04-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,3/13/2012,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
44770,49907,w[1118]; P{y[+t7.7] w[+mC]=GMR35D04-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,5/3/2012,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
45161,50300,w[1118]; P{y[+t7.7] w[+mC]=GMR47C12-GAL4}attP2...,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,5/15/2012,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
49349,54657,w[1118]; PBac{y[+mDint2] w[+mC]=GMR35D04-lexA}...,1;3,3,"89E11, 3R:17052863..17052863 (r6, Ti)",,1/27/2014,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses lexA under the control of DNA sequen...


In [18]:
#mask for the GMR lexA lines
lexa_mask = (search_table(bloomington,'Rubin') & search_table(bloomington,'lexA'))>0
#mask for the GMR Gal4 lines
gal4_mask = (search_table(bloomington,'Rubin') & search_table(bloomington,'GAL4'))>0
#get a list of the GMR id's for the first 100 lexA lines that are avalable
GMR_lexa_list = [s.split('GM')[1][:6] for s in bloomington[lexa_mask][0:100]['Genotype']]
GMR_lexa_list = [x for x in GMR_lexa_list if np.sum(search_table(strain_db,x))==0]
#now find the GAL4 equivelent for the first 100 lexA lines
gal4_order_masks = [bloomington['Genotype'].str.contains(gmr) for gmr in GMR_lexa_list]

In [19]:
gal4_order_mask = (np.sum(gal4_order_masks,axis = 0)>0) & gal4_mask
bloomington[gal4_order_mask]

,Stk #,Genotype,Ch # all,Ch TE{},Breakpts/Insertion,A.K.A.,Date added,Donor,Donor's source,Comments
41059,45006,w[1118]; P{y[+t7.7] w[+mC]=GMR10G12-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,3/14/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
41302,45394,w[1118]; P{y[+t7.7] w[+mC]=GMR14B12-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,4/5/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
42448,47318,w[1118]; P{y[+t7.7] w[+mC]=GMR11H07-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,9/20/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
42565,47452,w[1118]; P{y[+t7.7] w[+mC]=GMR11D01-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,9/20/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
42566,47453,w[1118]; P{y[+t7.7] w[+mC]=GMR11D08-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,9/20/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
42569,47456,w[1118]; P{y[+t7.7] w[+mC]=GMR12B03-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,9/20/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
42784,47842,w[1118]; P{y[+t7.7] w[+mC]=GMR10E08-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,11/18/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
42805,47865,w[1118]; P{y[+t7.7] w[+mC]=GMR15C03-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,11/18/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
42809,47869,w[1118]; P{y[+t7.7] w[+mC]=GMR16A04-GAL4}attP2...,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,11/18/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...
43142,48240,w[1118]; P{y[+t7.7] w[+mC]=GMR10B02-GAL4}attP2,1;3,3,"68A4, 3L:11070538..11070538 (r6, Ti)",,12/31/2011,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses GAL4 under the control of DNA sequen...


In [20]:
GMR_order_list = [s.split('GM')[1][:6] for s in bloomington[gal4_order_mask]['Genotype']]
#interesting - there appears to be a few lexA lines that don't have GAL4 equivelents, at least at bloomington
notinboth = set(GMR_lexa_list) - set(GMR_order_list)
bloomington[bloomington['Genotype'].str.contains(list(notinboth)[0])]

,Stk #,Genotype,Ch # all,Ch TE{},Breakpts/Insertion,A.K.A.,Date added,Donor,Donor's source,Comments
47289,52504,w[1118]; P{y[+t7.7] w[+mC]=GMR16D08-lexA}attP40,1;2,2,"25C6, 2L:5108448..5108448 (r6, Ti)",,12/2/2013,"Gerald M. Rubin, Janelia Farm/HHMI",,Expresses lexA under the control of DNA sequen...


In [21]:
#now double check to make sure we don't have these lines in our db
np.sum([np.sum(search_table(strain_db,x)) for x in GMR_order_list])

0

In [22]:
#bloomington id's for the lines to order
np.array(bloomington['Stk #'][gal4_order_mask])

array(['45006', '45394', '47318', '47452', '47453', '47456', '47842',
       '47865', '47869', '48240', '48246', '48247', '48250', '48251',
       '48275', '48276', '48279', '48436', '48440', '48441', '48442',
       '48445', '48447', '48452', '48455', '48464', '48465', '48469',
       '48474', '48480', '48487', '48494', '48497', '48504', '48506',
       '48523', '48525', '48534', '48537', '48541', '48548', '48556',
       '48560', '48565', '48566', '48569', '48571', '48575', '48577',
       '48579', '48587', '48588', '48595', '48597', '48602', '48605',
       '48606', '48647', '48661', '48664', '48667', '48671', '48672',
       '48675', '48676', '48682', '48683', '48684', '48687', '48689',
       '48702', '48706', '48710', '48716', '48720', '48727', '48731',
       '48738', '48739', '48744', '49236', '49240', '49247', '49255',
       '49257', '49258', '49261', '49826', '49827'], dtype=object)

In [23]:
order_me_mask = np.sum([search_table(bloomington,line) for line in order_me],axis = 0)>0

In [33]:
# order 9.4.2015
order_9_4_2015 = (order_me_mask & gal4_mask) | gal4_order_mask
for x in bloomington['Stk #'][order_9_4_2015]:
    print x

41329
45006
45394
45484
47241
47318
47452
47453
47456
47714
47842
47865
47869
48020
48240
48246
48247
48250
48251
48275
48276
48279
48436
48440
48441
48442
48445
48447
48452
48455
48464
48465
48469
48474
48480
48487
48494
48497
48504
48506
48523
48525
48534
48537
48541
48548
48556
48560
48565
48566
48569
48571
48575
48577
48579
48587
48588
48595
48597
48602
48605
48606
48647
48661
48664
48667
48671
48672
48675
48676
48682
48683
48684
48687
48689
48702
48706
48710
48716
48720
48727
48731
48738
48739
48744
49236
49240
49247
49255
49257
49258
49261
49411
49826
49827
49907
50300


In [28]:
bloomington[order_9_4_2015].to_csv('screen_part1.csv')

In [40]:
gn = [x[x.find('GMR'):x.find('GMR')+8] for x in bloomington['Genotype'][order_9_4_2015]]
bid = bloomington['Stk #'][order_9_4_2015]

In [49]:
import csv
with open('screen_part1.csv','wt') as f:
    csv.writer(f).writerows(np.vstack((gn,bid)).T)

In [33]:
# first part of screen - 10.1.15 - suggested lines already in database
# J-229 to J-246
jrange = range(229,246)
jstrings = ['J-%s'%(jnum) for jnum in jrange]

In [35]:
pilot_idx = strain_db['Balancer chromosomes'].str.contains(string.join(jstrings,'|'),na=False)
strain_db[pilot_idx]

,Balancer chromosomes,nickname,chromosome I,chromosome II,chromosome III,"source, date",notes
419,J-229,R38H06-GAL4 (III),w[1118],+,P{y[+t7.7] w[+mC]=GMR38H06-GAL4}attP2,Bloom 50029,Thad's Stock Request
420,J-230,R29H04-GAL4 (III),w[1118],+,P{y[+t7.7] w[+mC]=GMR29H04-GAL4}attP2,Bloom 49505,Thad's Stock Request
421,J-231,R81E05_GAL4 (III),w[1118],+,P{y[+t7.7] w[+mC]=GMR81E05_GAL4}attP2,Bloom 40115,Thad's Stock Request
422,J-232,R42D02_GAL4 (III),w[1118],+,P{y[+t7.7] w[+mC]=GMR42D02_GAL4}attP2,Bloom 41250,Thad's Stock Request
423,J-233,R87B09_GAL4 (III),w[1118],+,P{y[+t7.7] w[+mC]=GMR87B09_GAL4}attP2,Bloom 41317,Thad's Stock Request
424,J-234,R37A01_GAL4 (III),w[1118],+,P{y[+t7.7] w[+mC]=GMR37A01_GAL4}attP2,Bloom 45966,Thad's Stock Request
425,J-235,R27H05_GAL4 (III),w[1118],+,"P{y[+t7.7] w[+mC]=GMR27H05_\nGAL4}attP2/TM3, S...",Bloom 47519,Thad's Stock Request
426,J-236,R10F02_GAL4 (III),w[1118],+,P{y[+t7.7] w[+mC]=GMR10F02_GAL4}attP2,Bloom 47843,Thad's Stock Request
427,J-237,R30C07_GAL4 (III),w[1118],+,P{y[+t7.7] w[+mC]=GMR30C07_GAL4}attP2,Bloom 48097,Thad's Stock Request
428,J-238,R11B02_GAL4 (III),w[1118],+,P{y[+t7.7] w[+mC]=GMR11B02_GAL4}attP2,Bloom 48284,Thad's Stock Request
